```markdown
# Introducción

En este laboratorio, se implementará una primera versión de una aplicación cuya funcionalidad fue descrita en la Tarea 1. Esta primera versión se centrará en el cifrado y descifrado de información utilizando un criptosistema simétrico. Se recomienda el uso del estándar para criptografía simétrica, aunque los autores pueden optar por un criptosistema diferente y justificar su elección. También se puede utilizar más de un criptosistema simétrico, proporcionando la opción de seleccionarlo.

La tarea se dividirá en dos partes:

1. **Cifrado y Descifrado con Criptosistema Simétrico**: En esta parte, se implementarán las funciones necesarias para cifrar y descifrar archivos utilizando un criptosistema simétrico. Se incluirán funciones para generar claves, leer y escribir archivos, y manejar el padding necesario para el cifrado.

2. **Algoritmos de Clave Pública para Guardar Claves Simétricas**: En esta parte, se implementarán algoritmos de clave pública para proteger las claves secretas utilizadas en los algoritmos simétricos de la primera parte. Esto incluirá la generación de pares de claves públicas y privadas, así como el cifrado y descifrado de las claves simétricas.

El objetivo de este laboratorio es proporcionar una base sólida para la implementación de sistemas de cifrado y descifrado, utilizando tanto criptosistemas simétricos como de clave pública.
```

# Explicación de las Librerías Utilizadas

En el código proporcionado, se utilizan varias librerías de Python. A continuación, se explica cada una de ellas y su propósito:

## Librerías del Sistema Operativo

### `os`
La librería `os` proporciona una forma de usar funcionalidades dependientes del sistema operativo, como leer o escribir en el sistema de archivos.

## Librerías de Interfaz Gráfica

### `tkinter`
`tkinter` es la librería estándar de Python para crear interfaces gráficas de usuario (GUI). Permite crear ventanas, botones, cuadros de texto, etc.

### `filedialog` de `tkinter`
`filedialog` es un módulo dentro de `tkinter` que proporciona cuadros de diálogo estándar para abrir y guardar archivos. Es útil para permitir al usuario seleccionar archivos desde la interfaz gráfica.

## Librerías de Criptografía

### `Crypto.PublicKey.RSA`
Esta librería se utiliza para trabajar con el algoritmo RSA, que es un sistema de cifrado de clave pública. Permite generar claves RSA y realizar operaciones de cifrado y descifrado.

### `Crypto.Cipher.AES`
Esta librería se utiliza para trabajar con el algoritmo AES (Advanced Encryption Standard), que es un sistema de cifrado simétrico. Permite cifrar y descifrar datos utilizando una clave secreta.

### `Crypto.Cipher.PKCS1_OAEP`
Esta librería proporciona una implementación del esquema de cifrado OAEP (Optimal Asymmetric Encryption Padding) para RSA. Se utiliza para mejorar la seguridad del cifrado RSA.

### `Crypto.Util.Padding`
Este módulo proporciona funciones para añadir y eliminar padding (relleno) a los datos. El padding es necesario en algunos modos de cifrado para asegurar que los datos tengan la longitud correcta.

### `Crypto.Random.get_random_bytes`
Esta función se utiliza para generar bytes aleatorios, que son necesarios para crear claves y vectores de inicialización (IV) seguros en criptografía.

## Resumen

Estas librerías se combinan para crear una aplicación que puede:
- Interactuar con el sistema de archivos.
- Proporcionar una interfaz gráfica para el usuario.
- Realizar operaciones de cifrado y descifrado utilizando algoritmos seguros como RSA y AES.

In [1]:
import os
import tkinter as tk
from tkinter import filedialog
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes

```markdown
## Funciones para Leer y Escribir Archivos

### `leer_archivo(ruta_archivo)`

Esta función se encarga de leer el contenido de un archivo y devolverlo. Toma como parámetro `ruta_archivo`, que es la ruta del archivo que se desea leer. La función abre el archivo en modo binario (`'rb'`), lee su contenido y lo retorna.

### `escribir_archivo(ruta_archivo, datos)`

Esta función se utiliza para escribir datos en un archivo. Toma dos parámetros: `ruta_archivo`, que es la ruta del archivo donde se desean escribir los datos, y `datos`, que es el contenido que se desea escribir en el archivo. La función abre el archivo en modo binario (`'wb'`) y escribe los datos proporcionados en él.
```

In [2]:
# Función para leer el archivo y devolver su contenido
def leer_archivo(ruta_archivo):
    with open(ruta_archivo, 'rb') as f:
        return f.read()

# Función para escribir datos en un archivo
def escribir_archivo(ruta_archivo, datos):
    with open(ruta_archivo, 'wb') as f:
        f.write(datos)


```markdown
## Función `cifrar_archivo`

La función `cifrar_archivo` cifra un archivo utilizando AES en modo CBC. A continuación se describe el proceso:

1. **Leer el Archivo**: Utiliza `leer_archivo` para obtener el contenido del archivo.
2. **Generar Clave**: Crea una clave de cifrado con `generar_clave`.
3. **Crear Cifrador AES**: Inicializa un cifrador AES en modo CBC y obtiene el IV.
4. **Añadir Padding**: Ajusta los datos para que sean múltiplos de 16 bytes.
5. **Cifrar Datos**: Cifra los datos ajustados.
6. **Guardar IV y Datos Cifrados**: Almacena el IV y los datos cifrados en el archivo de salida.
7. **Guardar Clave**: Guarda la clave en un archivo de claves con `guardar_clave`.

### Parámetros

- `ruta_archivo`: Ruta del archivo a cifrar.
- `clave_tamaño`: Tamaño de la clave (16, 24 o 32 bytes).
- `ruta_salida`: Ruta del archivo cifrado.
- `ruta_claves`: Ruta del archivo de claves.

In [3]:
def cifrar_archivo(ruta_archivo, clave_tamaño, ruta_salida, ruta_claves):
    datos = leer_archivo(ruta_archivo)  # Leer el archivo a cifrar
    clave = generar_clave(clave_tamaño)
    print("Clave generada:", clave)
    # Crear un cifrador AES en modo CBC
    cipher = AES.new(clave, AES.MODE_CBC)
    iv = cipher.iv  # Inicializar el vector de inicialización (IV)

    # Añadir padding (relleno) a los datos para que sean múltiplos del tamaño de bloque (16 bytes)
    datos_padded = pad(datos, AES.block_size)

    # Cifrar los datos
    ciphertext = cipher.encrypt(datos_padded)

    nombre_archivo, extension = os.path.splitext(os.path.basename(ruta_archivo))

    # Guardar el IV y el texto cifrado en el archivo de salida
    with open(ruta_salida, 'wb') as f:
        f.write(iv)           # Guardar el IV (16 bytes)
        f.write(ciphertext)    # Guardar el texto cifrado
    guardar_clave(nombre_archivo, clave, extension, ruta_claves)
    print(f"Archivo cifrado guardado en {ruta_salida}")


```markdown
## Función `descifrar_archivo`

La función `descifrar_archivo` descifra un archivo cifrado utilizando AES en modo CBC. A continuación se describe el proceso:

1. **Leer Claves**: Utiliza `leer_claves` para obtener la lista de claves almacenadas.
2. **Leer el Archivo Cifrado**: Abre el archivo cifrado y lee el IV (16 bytes) y el texto cifrado restante.
3. **Obtener Clave y Extensión**: Extrae la clave y la extensión del archivo original de la lista de claves.
4. **Seleccionar Ruta de Salida**: Utiliza un cuadro de diálogo para seleccionar la ruta donde se guardará el archivo descifrado.
5. **Crear Descifrador AES**: Inicializa un descifrador AES en modo CBC con el IV leído.
6. **Descifrar Datos**: Descifra los datos y elimina el padding.
7. **Guardar Archivo Descifrado**: Escribe los datos descifrados en el archivo de salida.

### Parámetros

- `ruta_archivo`: Ruta del archivo cifrado.
- `ruta_claves`: Ruta del archivo que contiene las claves.

### Excepciones

- `ValueError`: Se lanza si el padding es incorrecto o los datos están corruptos.
```

In [4]:
def descifrar_archivo(ruta_archivo, ruta_claves):
    lista_claves = leer_claves(ruta_claves)
    with open(ruta_archivo, 'rb') as f:
        iv = f.read(16)        # Leer el IV (16 bytes)
        ciphertext = f.read()  # Leer el texto cifrado restante

    nombre_archivo, extension = os.path.splitext(os.path.basename(ruta_archivo))
    
    if nombre_archivo in lista_claves:
        clave = lista_claves[nombre_archivo][0]
        extension = lista_claves[nombre_archivo][1]
        print(f"La clave para '{nombre_archivo}' es : {clave}")

    ruta_salida = filedialog.asksaveasfilename(defaultextension=extension,
                                               initialfile=nombre_archivo,
                                               filetypes=[("Todos los archivos", "*.*")])
    
    # Crear un descifrador AES en modo CBC con el mismo IV
    cipher = AES.new(clave, AES.MODE_CBC, iv=iv)

    # Descifrar los datos
    datos_padded = cipher.decrypt(ciphertext)

    try:
        # Eliminar el padding de los datos
        datos = unpad(datos_padded, AES.block_size)

        # Escribir el archivo descifrado
        escribir_archivo(ruta_salida, datos)
        print(f"Archivo descifrado guardado en {ruta_salida}")
    except ValueError:
        print("Error: el padding es incorrecto o los datos están corruptos.")


```markdown
## Función `generar_clave`

La función `generar_clave` se utiliza para crear una clave de cifrado de un tamaño específico. A continuación se describe el proceso:

1. **Seleccionar Tamaño de Clave**: La función toma un parámetro `clave_tamaño` que puede ser '32', '24' o '16', representando el tamaño de la clave en bytes.
2. **Generar Clave**: Dependiendo del tamaño seleccionado, la función genera una clave aleatoria utilizando `get_random_bytes`.
3. **Validar Opción**: Si el tamaño de la clave no es válido, se imprime un mensaje de error.

### Parámetros

- `clave_tamaño`: Tamaño de la clave en bytes ('32' para 256 bits, '24' para 192 bits, '16' para 128 bits).

### Retorno

- `clave`: La clave generada de tamaño especificado.

In [5]:
def generar_clave(clave_tamaño):
    if clave_tamaño == '32':
        clave = get_random_bytes(32)
    elif clave_tamaño == '24':
        clave = get_random_bytes(24)
    elif clave_tamaño == '16':
        clave = get_random_bytes(16)
    else:
        print("Opción no válida")
    return clave # AES-256 usa una clave de 256 bits (32 bytes)


```markdown
## Función `guardar_clave`

La función `guardar_clave` se utiliza para almacenar una clave de cifrado junto con el nombre del archivo y su extensión en un archivo binario. A continuación se describe el proceso:

1. **Leer Claves Existentes**: Utiliza la función `leer_claves` para obtener las claves almacenadas previamente en el archivo especificado por `ruta_claves`.

2. **Actualizar o Añadir Clave**:
    - Si el nombre del archivo (`nombre`) ya existe en la lista de claves, actualiza la clave y la extensión asociada.
    - Si el nombre del archivo no existe, añade una nueva entrada con el nombre, la clave y la extensión.

3. **Guardar Claves en el Archivo**:
    - Si se actualiza una clave existente, abre el archivo en modo binario de escritura (`'wb'`) y reescribe todas las claves.
    - Si se añade una nueva clave, abre el archivo en modo binario de adición (`'ab'`) y añade la nueva clave al final del archivo.

### Parámetros

- `nombre`: Nombre del archivo asociado a la clave.
- `clave`: Clave de cifrado a almacenar.
- `extension`: Extensión del archivo original.
- `ruta_claves`: Ruta del archivo donde se almacenan las claves.

### Detalles del Proceso

- **Codificación**: Los nombres y extensiones de los archivos se codifican en UTF-8 antes de ser escritos en el archivo.
- **Longitud de los Datos**: Antes de escribir cada dato (nombre, clave, extensión), se escribe su longitud en un byte para facilitar la lectura posterior.

In [6]:
def guardar_clave(nombre, clave, extension, ruta_claves):
    lista_claves = leer_claves(ruta_claves)
    if nombre in lista_claves:
        lista_claves[nombre] = (clave, extension)
        with open(ruta_claves,'wb') as f:
            for name in lista_claves:
                name_bytes = name.encode('utf-8')
                f.write(len(name_bytes).to_bytes(1,'big'))
                f.write(name_bytes)
                
                f.write(len(lista_claves[name][0]).to_bytes(1, 'big'))
                f.write(lista_claves[name][0])

                ext_bytes = lista_claves[name][1].encode('utf-8')
                f.write(len(ext_bytes).to_bytes(1,'big'))
                f.write(ext_bytes)
    else:
        with open(ruta_claves,'ab') as f:
            nombre_bytes = nombre.encode('utf-8')
            f.write(len(nombre_bytes).to_bytes(1,'big'))
            f.write(nombre_bytes)
            
            f.write(len(clave).to_bytes(1,'big'))
            f.write(clave)

            extension_bytes = extension.encode('utf-8')
            f.write(len(extension_bytes).to_bytes(1,'big'))
            f.write(extension_bytes)
            

```markdown
## Función `leer_claves`

La función `leer_claves` lee un archivo binario con claves de cifrado y sus extensiones asociadas.

### Proceso

1. **Inicializar Diccionario**: Crea un diccionario vacío `claves_leidas`.
2. **Abrir Archivo**: Intenta abrir el archivo en modo binario de lectura (`'rb'`).
3. **Leer Datos**: Lee la longitud (1 byte) y luego el nombre, clave y extensión.
4. **Almacenar Claves**: Guarda el nombre, la clave y la extensión en `claves_leidas`.
5. **Excepciones**: Si el archivo no se encuentra (`FileNotFoundError`), retorna el diccionario vacío.

### Parámetros

- `ruta_claves`: Ruta del archivo con las claves.

### Retorno

- `claves_leidas`: Diccionario con las claves leídas.


In [7]:
def leer_claves(ruta_claves):
    claves_leidas = {}
    try:
        with open(ruta_claves,'rb') as f:
                while True:
                    len_nombre = f.read(1)
                    if not len_nombre:
                        break
                    len_nombre = int.from_bytes(len_nombre,'big')
                    nombre = f.read(len_nombre).decode('utf-8')
                
                    len_clave = f.read(1)
                    if not len_clave:
                        break
                    len_clave = int.from_bytes(len_clave,'big')
                    clave = f.read(len_clave)

                    len_extension = f.read(1)
                    if not len_extension:
                        break
                    len_extension = int.from_bytes(len_extension,'big')
                    extension = f.read(len_extension).decode('utf-8')
                
                    claves_leidas[nombre] = (clave, extension)
                return claves_leidas
    except FileNotFoundError:
        return claves_leidas

```markdown
## Función `crear_RSA`

La función `crear_RSA` genera un par de claves RSA (pública y privada) y las guarda en archivos separados.

### Proceso

1. **Generar Claves**: Utiliza `RSA.generate(2048)` para crear una clave RSA de 2048 bits.
2. **Exportar Claves**: Exporta la clave privada y la clave pública utilizando `export_key()`.
3. **Determinar Rutas de Archivos**:
    - Obtiene el directorio del archivo cifrado (`ruta_archivo_cifrado`) utilizando `os.path.dirname`.
    - Crea las rutas completas para los archivos de clave privada (`private.pem`) y clave pública (`public.pem`) utilizando `os.path.join` y `os.path.normpath`.
4. **Guardar Claves en Archivos**:
    - Abre el archivo de clave privada en modo binario de escritura (`'wb'`) y escribe la clave privada.
    - Abre el archivo de clave pública en modo binario de escritura (`'wb'`) y escribe la clave pública.

### Parámetros

- `ruta_archivo_cifrado`: Ruta del archivo cifrado, utilizada para determinar el directorio donde se guardarán las claves.

### Detalles del Proceso

- **Clave Privada**: Se guarda en un archivo llamado `private.pem` en el mismo directorio que el archivo cifrado.
- **Clave Pública**: Se guarda en un archivo llamado `public.pem` en el mismo directorio que el archivo cifrado.
```

In [8]:
def crear_RSA(ruta_archivo_cifrado):
    key = RSA.generate(2048)
    private_key = key.export_key()
    public_key = key.publickey().export_key()

    ruta_cortada = os.path.dirname(ruta_archivo_cifrado)
    ruta_privada = os.path.normpath(os.path.join(ruta_cortada, "private.pem"))
    ruta_publica = os.path.normpath(os.path.join(ruta_cortada, "public.pem"))
    
    with open(ruta_privada, "wb") as f:
        f.write(private_key)
    with open(ruta_publica, "wb") as f:
        f.write(public_key)
    

```markdown
## Función `habilitar_claves`

La función `habilitar_claves` se utiliza para desencriptar un archivo de claves utilizando una clave privada RSA.

### Proceso

1. **Solicitar Clave Privada**: Muestra un mensaje solicitando la clave privada para desencriptar el archivo de claves.
2. **Seleccionar Archivo de Clave Privada**: Abre un cuadro de diálogo para que el usuario seleccione el archivo de clave privada (`.pem`).
3. **Leer Clave Privada**: Lee el contenido del archivo seleccionado e importa la clave privada RSA.
4. **Crear Descifrador RSA**: Inicializa un descifrador RSA utilizando `PKCS1_OAEP` con la clave privada.
5. **Leer Archivo de Claves Cifrado**: Abre y lee el contenido del archivo de claves cifrado.
6. **Desencriptar Datos**: Utiliza el descifrador RSA para desencriptar el contenido del archivo de claves.
7. **Reescribir Archivo de Claves**: Sobrescribe el archivo de claves con los datos desencriptados.

### Parámetros

- `ruta_archivo_cifrado`: Ruta del archivo de claves cifrado.

### Retorno

- `decrypted_data`: Datos desencriptados del archivo de claves.
```

In [9]:
def habilitar_claves(ruta_archivo_cifrado):
    print("Se necesita que se facilite la clave privada para desencriptar el archivo de claves")
    root = tk.Tk()
    root.withdraw()  # Oculta la ventana principal de Tkinter
    ruta_archivo = filedialog.askopenfilename(
                title="Seleccionar archivo",
                filetypes=[("Todos los archivos", "*.pem")] #Seleccionamos el archivo clave privada
            )
    with open(ruta_archivo, "rb") as f:
        private_key = RSA.import_key(f.read()) #Leemos su contenido y lo importamos como clave RSA
        
    cipher_rsa = PKCS1_OAEP.new(private_key) #Creamos el desencriptador con dicha clave privaad

    with open(ruta_archivo_cifrado, "rb") as f:
        encrypted_data = f.read() #Leemos el archivo de claves.bin 

    decrypted_data = cipher_rsa.decrypt(encrypted_data) #Desencriptamos su contenido

    with open(ruta_archivo_cifrado, "wb") as f:
        f.write(decrypted_data) #Reescribimos el archivo de claves.bin, ahora desencriptado, en su mismo archivo

    return decrypted_data

```markdown
## Función `encriptar_claves`

La función `encriptar_claves` cifra un archivo de claves utilizando una clave pública RSA.

### Proceso

1. **Seleccionar Opción**: El usuario elige entre crear un nuevo par de claves RSA o usar una clave pública existente.
2. **Crear Nuevas Claves**:
    - Genera un nuevo par de claves RSA con `crear_RSA`.
    - Lee e importa la clave pública generada.
    - Cifra el archivo de claves con la clave pública y guarda los datos cifrados.
3. **Usar Clave Pública Propia**:
    - El usuario selecciona un archivo de clave pública (`.pem`).
    - Lee e importa la clave pública seleccionada.
    - Cifra el archivo de claves con la clave pública y guarda los datos cifrados.
4. **Validar Opción**: Muestra un mensaje de error si la opción no es válida.

### Parámetros

- `ruta_archivo_cifrado`: Ruta del archivo de claves a cifrar.

### Detalles

- **Clave Pública**: Utilizada para cifrar el archivo de claves.
- **Clave Privada**: Generada junto con la clave pública si se elige crear nuevas claves.
- **Archivo de Claves**: El archivo de claves se cifra y guarda en la misma ruta.


In [10]:
def encriptar_claves(ruta_archivo_cifrado):
    print("¿Desea crear un nuevo par de claves pública y privada, o desea utilizar una clave pública propia?")
    print("1. Crear nuevas")
    print("2. Utilizar mi clave")
    opcion = input("Selecciona una opción: ")
    if opcion == '1':
        crear_RSA(ruta_archivo_cifrado) #Creamos las claves pública y privada en el mismo directorio que el archivo claves.bin
        
        ruta_cortada = os.path.dirname(ruta_archivo_cifrado)
        ruta_archivo = os.path.normpath(os.path.join(ruta_cortada, "public.pem"))
        
        with open(ruta_archivo, "rb") as f:
            public_key = RSA.import_key(f.read()) #Leemos su contenido y lo importamos como clave RSA

        cipher_rsa = PKCS1_OAEP.new(public_key)

        with open(ruta_archivo_cifrado, "rb") as f:
            datos = f.read()
            
        encrypted_data = cipher_rsa.encrypt(datos)

        with open(ruta_archivo_cifrado, "wb") as f:
            f.write(encrypted_data)
            
    elif opcion == '2':
        print("Necesitamos que nos facilite la clave publica para encriptar el archivo de claves")
        root = tk.Tk()
        root.withdraw()  # Oculta la ventana principal de Tkinter
        ruta_archivo = filedialog.askopenfilename(
                title="Seleccionar archivo",
                filetypes=[("Todos los archivos", "*.pem")] #Seleccionamos el archivo clave publica
            )
        with open(ruta_archivo, "rb") as f:
            public_key = RSA.import_key(f.read()) #Leemos su contenido y lo importamos como clave RSA

        cipher_rsa = PKCS1_OAEP.new(public_key)

        with open(ruta_archivo_cifrado, "rb") as f:
            datos = f.read()
            
        encrypted_data = cipher_rsa.encrypt(datos)

        with open(ruta_archivo_cifrado, "wb") as f:
            f.write(encrypted_data)
    else:
        print("Opción no válida. Inténtalo de nuevo.")

```markdown
## Menú de la Aplicación

El menú de la aplicación permite al usuario seleccionar entre varias opciones para cifrar y descifrar archivos, así como gestionar las claves de cifrado. A continuación se describen las opciones disponibles en el menú:

1. **Cifrar un archivo**: Esta opción permite al usuario seleccionar un archivo y cifrarlo utilizando una clave simétrica. El usuario debe proporcionar el tamaño de la clave (32, 24 o 16 bytes). El archivo cifrado se guarda en una ubicación especificada por el usuario.

2. **Descifrar un archivo**: Esta opción permite al usuario seleccionar un archivo cifrado y descifrarlo utilizando la clave correspondiente. El archivo descifrado se guarda en una ubicación especificada por el usuario.

3. **Cargar archivo de claves**: Esta opción permite al usuario cargar un archivo de claves existente o crear uno nuevo. Si el archivo de claves está cifrado con una clave pública, el usuario debe proporcionar la clave privada para descifrarlo.

4. **Encriptar claves**: Esta opción permite al usuario cifrar el archivo de claves utilizando una clave pública RSA. El usuario puede optar por crear un nuevo par de claves RSA o utilizar una clave pública existente.

5. **Salir**: Esta opción permite al usuario salir de la aplicación.

El menú se presenta al usuario en un bucle, permitiendo realizar múltiples operaciones hasta que el usuario decida salir.
```

In [11]:
def menu():
    print("1. Cifrar un archivo")
    print("2. Descifrar un archivo")
    print("3. Cargar archivo de claves (HACER LO PRIMERO SI YA TIENE CLAVES O PARA CREAR ARCHIVO DE CLAVES)")
    print("4. Encriptar claves (HACER LO ULTIMO SI QUIERES ENCRIPTAR LAS CLAVES")
    print("5. Salir")
    return input("Selecciona una opción: ")


In [12]:
def main():
    lista_claves = {}
    ruta_salida = ""
    while True:
        opcion = menu()
        if opcion == '1':
            # Cifrar archivo
            root = tk.Tk()
            root.withdraw()  # Oculta la ventana principal de Tkinter
            ruta_archivo = filedialog.askopenfilename(
                title="Seleccionar archivo",
                filetypes=[("Todos los archivos", ".*")]
            )
            
            if ruta_archivo:
                print(f"Has seleccionado: {ruta_archivo}")
                clave = input("Introduce tamaño de clave: 32 (256), 24 (192), 16 (128)")
                nombre_archivo, extension = os.path.splitext(os.path.basename(ruta_archivo))
                ruta_salida = filedialog.asksaveasfilename(defaultextension=".bin",
                                                           initialfile=nombre_archivo,
                                                           filetypes=[("Archivos BIN", "*.bin"),("Todos los archivos", "*.*")])
                cifrar_archivo(ruta_archivo, clave, ruta_salida, ruta_claves)
                print(f"Archivo cifrado y guardado en: {ruta_salida}")
            else:
                print("No se seleccionó ningún archivo.")
                
        elif opcion == '2':
            # Descifrar archivo
            root = tk.Tk()
            root.withdraw()  # Oculta la ventana principal de Tkinter
            ruta_archivo = filedialog.askopenfilename(
                title="Seleccionar archivo cifrado",
                filetypes=[("Todos los archivos", "*.bin")]
            )
            
            if ruta_archivo:
                print(f"Has seleccionado: {ruta_archivo}")
                descifrar_archivo(ruta_archivo, ruta_claves)
            else:
                print("No se seleccionó ningún archivo.")
        elif opcion == '3':
            root = tk.Tk()
            root.withdraw()  # Oculta la ventana principal de Tkinter
            ruta_archivo = filedialog.askopenfilename(
                title="Seleccionar archivo de claves",
                filetypes=[("Todos los archivos", "*.bin")]
            )
            
            if ruta_archivo:
                ruta_claves = ruta_archivo
                print("¿Este archivo está encriptado clave pública?")
                result = input("Seleccione 1 (SI) o 2 (NO)")
                if result == '1':
                    lista_claves = habilitar_claves(ruta_claves)
            else:
                print("No se seleccionó ningún archivo. Crearemos un archivo de claves nuevo...")
                ruta_claves = filedialog.asksaveasfilename(defaultextension=".bin",
                                                           initialfile="claves",
                                                           filetypes=[("Archivos BIN", "*.bin"),("Todos los archivos", "*.*")])
                with open(ruta_claves, 'wb') as f:
                    pass
        elif opcion == '4':
            encriptar_claves(ruta_claves)
        elif opcion == '5':
            print("Saliendo...")
            break
        else:
            print("Opción no válida. Inténtalo de nuevo.")


In [13]:
if __name__ == "__main__":
    main()


1. Cifrar un archivo
2. Descifrar un archivo
3. Cargar archivo de claves (HACER LO PRIMERO SI YA TIENE CLAVES O PARA CREAR ARCHIVO DE CLAVES)
4. Encriptar claves (HACER LO ULTIMO SI QUIERES ENCRIPTAR LAS CLAVES
5. Salir
No se seleccionó ningún archivo.
1. Cifrar un archivo
2. Descifrar un archivo
3. Cargar archivo de claves (HACER LO PRIMERO SI YA TIENE CLAVES O PARA CREAR ARCHIVO DE CLAVES)
4. Encriptar claves (HACER LO ULTIMO SI QUIERES ENCRIPTAR LAS CLAVES
5. Salir
Saliendo...
